# Portfolio Optimization

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.insert(1, './..')

In [2]:
from quantML import scrape
from quantML import optimize

import scipy
import numpy as np
import pandas as pd
from scipy.optimize import minimize

## Fake Data

In [3]:
tickers = np.array(scrape.get_sp500_tickers()[0:400:20])
price_df = scrape.get_prices(tickers, '2015-01-01', '2020-12-21',)
means = price_df.groupby('ticker').mean()
tickers = np.array(means.index)

INFO:quantML.scrape:Scraping MMM - 1 out of 20 tickers
INFO:quantML.scrape:Scraping LNT - 2 out of 20 tickers
INFO:quantML.scrape:Scraping ANSS - 3 out of 20 tickers
INFO:quantML.scrape:Scraping BLL - 4 out of 20 tickers
INFO:quantML.scrape:Scraping COG - 5 out of 20 tickers
INFO:quantML.scrape:Scraping CVX - 6 out of 20 tickers
INFO:quantML.scrape:Scraping CXO - 7 out of 20 tickers
INFO:quantML.scrape:Scraping DVN - 8 out of 20 tickers
INFO:quantML.scrape:Scraping ETN - 9 out of 20 tickers
INFO:quantML.scrape:Scraping EXPD - 10 out of 20 tickers
INFO:quantML.scrape:Scraping FBHS - 11 out of 20 tickers
INFO:quantML.scrape:Scraping HIG - 12 out of 20 tickers
INFO:quantML.scrape:Scraping IEX - 13 out of 20 tickers
INFO:quantML.scrape:Scraping J - 14 out of 20 tickers
INFO:quantML.scrape:Scraping LRCX - 15 out of 20 tickers
INFO:quantML.scrape:Scraping MAR - 16 out of 20 tickers
INFO:quantML.scrape:Scraping MDLZ - 17 out of 20 tickers
INFO:quantML.scrape:Scraping NTRS - 18 out of 20 ticke

In [4]:
pred_move = np.random.normal(0.0, 0.01, size=(tickers.shape))*100
costs = means.to_numpy().T[0]
positions = np.random.uniform(0,1,size=(tickers.shape))
positions = positions / np.sum(positions)

## Formulation

Objective Function - Maximize expected return while maintaining diversity in stocks

Constraints - Bounded percentages, constrained total positions

In [5]:
corr = price_df.pivot(index='date', columns='ticker', values='close').corr().to_numpy()

def get_adjusted_correlation(corr, positions):
    return scipy.linalg.norm(np.triu(corr)@positions, ord=2)

def objective(x, pred_move=pred_move):
    positions = x
    expected_return = pred_move@positions
    adjusted_correlation = get_adjusted_correlation(corr, positions)
    return -(expected_return / adjusted_correlation / np.std(positions))

bnds = [(0,1) for i in pred_move] # no short positions
cons = ({'type': 'eq', 'fun': lambda x:  1-sum(x)},) # fractions add up to 0

## Implementation

In [6]:
res = minimize(objective, positions, method='SLSQP', constraints=cons, bounds=bnds, options={'maxiter': 10000})
final = pd.DataFrame(costs); final.columns = ['cost']; final['ml_pred_weekly_movement'] = pred_move
final['opt_fractional_position'] = res.x; final.index = tickers
final.loc[final.ml_pred_weekly_movement < 0, 'opt_fractional_position'] = 0
final['opt_fractional_position'] = final['opt_fractional_position'] / final['opt_fractional_position'].sum()

print(f'Status - {res.success}\n')
print('Original')
print(f'Expected Return: {pred_move@positions}')
print(f'Diversification: {scipy.linalg.norm(np.triu(corr)@positions)}')
print()
print('Model Final')
print(f'Expected Return: {pred_move@res.x}')
print(f'Diversification: {scipy.linalg.norm(np.triu(corr)@res.x)}')
print()
print('ML Adjusted Final')
r = pred_move@final['opt_fractional_position']
print(f'Expected Return: {r}')
d = scipy.linalg.norm(np.triu(corr)@final['opt_fractional_position'])
print(f'Diversification: {d}')

final.sort_values(by='opt_fractional_position', ascending=False)

Status - True

Original
Expected Return: -0.0263951624060954
Diversification: 1.1499058121557857

Model Final
Expected Return: 1.1329151735605247
Diversification: 0.45799028984081575

ML Adjusted Final
Expected Return: 1.2017728507176098
Diversification: 0.46913199693665514


,cost,ml_pred_weekly_movement,opt_fractional_position
COG,23.062746,1.882418,2.777954e-01
ANSS,159.431011,1.180008,1.966983e-01
BLL,48.780243,0.886481,1.727404e-01
FBHS,57.457926,1.247746,1.472019e-01
DVN,33.907560,0.533263,1.350094e-01
LNT,41.953404,0.844951,3.714131e-02
EXPD,62.784714,0.078011,2.735141e-02
PEG,49.110864,0.730255,6.061864e-03
LRCX,171.296410,0.419036,2.321576e-12
NTRS,85.709136,-1.041542,0.000000e+00
